<a href="https://colab.research.google.com/github/yuri-spizhovyi-mit/housing-insights-risk-dashboard/blob/main/ml/notebooks/forecasting_analysis.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Housing Insights & Risk Dashboard  
## Data Engineering & Forecasting Notebook  
### Models: ARIMA, Prophet, LSTM
#### Author: Yuri Spizhovyi
#### Environment: Google Colab + Python + Pandas + Statsmodels + TensorFlow
#### Objective:
- Load datasets (HPI, rent, demographics, macro, metrics)
- Explore trends, seasonality, missingness
- Define feature engineering strategy
- Prepare feature tables for ARIMA, Prophet, LSTM

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("seaborn-v0_8")
pd.set_option("display.max_columns", None)

In [ ]:
!git clone https://github.com/yuri-spizhovyi-mit/housing-insights-risk-dashboard.git
%cd housing-insights-risk-dashboard/data/raw

In [ ]:
# Load datasets
df_hpi = pd.read_csv("house_price_index.csv")
df_rent = pd.read_csv("rent_index.csv")
df_demo = pd.read_csv("demographics.csv")
df_macro = pd.read_csv("macro_economic_data.csv")
df_metrics = pd.read_csv("metrics.csv")

dfs = {
    "house_price_index": df_hpi,
    "rent_index": df_rent,
    "demographics": df_demo,
    "macro_economic": df_macro,
    "metrics": df_metrics,
}

for name, df in dfs.items():
    print(f"\n===== {name.upper()} =====")
    print(df.head())
    print(df.info())

In [ ]:
for name, df in dfs.items():
    if "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"])
        df.sort_values("date", inplace=True)
        dfs[name] = df

print("Date columns converted and sorted.")

In [ ]:
for name, df in dfs.items():
    print(f"\n{name}:")
    print("Date range:", df["date"].min(), "→", df["date"].max())
    print("Missing values:\n", df.isna().sum())

In [ ]:
# Plot HPI trend
plt.figure(figsize=(12, 5))
sns.lineplot(data=df_hpi, x="date", y="benchmark_price")
plt.title("House Price Index Trend")
plt.show()

In [ ]:
# Plot rent trend
plt.figure(figsize=(12, 5))
sns.lineplot(data=df_rent, x="date", y="rent_value")
plt.title("Rent Index Trend")
plt.show()

SECTION 2

Exploratory Data Analysis (EDA) + Feature Engineering Plan**    

In [ ]:
#Imports for EDA
%pip install statsmodels

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import STL


In [ ]:
# Detect Stationarity (ADF Test) - ARIMA and partially Prophet

In [ ]:
def adf_test(series, name="Series"):
    print(f"\nADF Test for {name}")
    result = adfuller(series.dropna())
    labels = ["ADF statistic", "p-value", "# lags used", "# observations"]
    out = pd.Series(result[0:4], index=labels)
    for key, val in result[4].items():
        out[f"critical value ({key})"] = val
    print(out)
    if result[1] <= 0.05:
        print("=> Stationary")
    else:
        print("=> Non-stationary")

adf_test(df_hpi["benchmark_price"], "House Price Index")
adf_test(df_rent["rent_value"], "Rent Index")


Interpretation rules (AI will do this for you in Colab):

    p-value > 0.05 → non-stationary → ARIMA requires differencing

    p-value < 0.05 → stationary → ARIMA can train directly

Housing prices and rents are almost always non-stationary.

In [ ]:
# Detect Seasonality (STL Decomposition)
from matplotlib import pyplot as plt

def stl_plot(df, value_col, title):
    # Ensure the date column is the index and it's unique for resampling
    # Filtering is handled before calling this function
    df = df.set_index("date")
    
    # Ensure only the value column is considered for asfreq and STL
    df = df[[value_col]].asfreq("MS")  # monthly start frequency
    
    stl = STL(df[value_col], robust=True)
    res = stl.fit()

    fig = res.plot()
    fig.set_size_inches(12, 8)
    fig.suptitle(title, fontsize=16)
    plt.show()

# Filter df_hpi to get a single time series (e.g., Calgary, All property types)
df_hpi_filtered = df_hpi[(df_hpi['city'] == 'Calgary') & (df_hpi['property_type'] == 'All')].copy()
stl_plot(df_hpi_filtered, "benchmark_price", "STL Decomposition – HPI (Calgary, All)")

# Filter df_rent to get a single time series (e.g., Calgary)
df_rent_filtered = df_rent[df_rent['city'] == 'Calgary'].copy()
stl_plot(df_rent_filtered, "rent_value", "STL Decomposition – Rent Index (Calgary)")

This reveals:

long-term trend

seasonal cycle

residual (noise)

LSTM and Prophet use this implicitly.
ARIMA needs you to remove it manually using differencing.

4. Correlation With Macro & Demographics

In [ ]:
# Merge macro + HPI on date. Clarify which variables may help LSTM & Prophet.
df_hpi_merged = df_hpi.merge(df_macro, on="date", how="left")

plt.figure(figsize=(14,6))
sns.heatmap(df_hpi_merged.corr(numeric_only=True), annot=False, cmap="coolwarm")
plt.title("Correlation Heatmap (HPI + Macro)")
plt.show()

In [ ]:
# the same for rent + macro:
df_rent_merged = df_rent.merge(df_macro, on="date", how="left")

plt.figure(figsize=(14,6))
sns.heatmap(df_rent_merged.corr(numeric_only=True), annot=False, cmap="coolwarm")
plt.title("Correlation Heatmap (Rent + Macro)")
plt.show()


This reveals drivers like:

interest rate

unemployment

CPI

GDP growth

population growth

migration flows

These are critical features for LSTM forecasting.

# Feature Engineering Strategy

## 1. ARIMA (Univariate Time Series)
ARIMA and SARIMA operate on a single, stationary time series.  
They do not require or benefit from external regressors.

**Characteristics**
- Uses only the target variable (price or rent).
- Requires:
  - First-order differencing to remove trend.
  - Seasonal differencing (12-month) for yearly patterns.
- Sensitive to missing values; series must be complete and evenly spaced.

**Recommended Feature Table**
| date | value |
Where:
- `date` is a monthly timestamp (`MS`)
- `value` is the target series (e.g., `benchmark_price`, `rent_value`)

---

## 2. Prophet (Trend + Seasonality + Optional Regressors)
Prophet is designed for business time-series with trend, seasonality, and structural breaks.

**Strengths**
- Automatically models yearly, weekly, and daily seasonality.
- Handles trend shifts using changepoints.
- Accepts external regressors as additive components.

**Useful Regressors**
- Interest rate
- CPI inflation (YoY)
- Unemployment
- Mortgage rate
- Population or migration (optional)

**Recommended Feature Table**
| ds | y | regressor_1 | regressor_2 | ... |
Where:
- `ds` = date column
- `y` = target series  
- Additional columns = macro or demographic indicators

---

## 3. LSTM (Multivariate Deep Learning)
LSTM models learn temporal patterns from multiple correlated variables.

**Strengths**
- Handles nonlinear relationships.
- Learns long-range dependencies.
- Uses rich feature sets (macro, demographics, anomalies, lag features).

**Recommended Inputs**
- Macro variables (GDP growth, CPI, rates)
- Demographics (population, income, migration)
- Listings activity / supply indicators
- Risk anomaly signals
- Engineered features:
  - lag-1, lag-3, lag-6, lag-12
  - rolling mean (3, 6, 12 months)
  - rolling std
  - month or seasonal dummy variables
  - YoY percentage change

**Recommended Feature Table**
| date | price | rent | interest_rate | cpi_yoy | population | migration_rate | ... |
Where additional columns can be added as needed for model performance.



In [ ]:
# ============================================================
# Rent Preprocessing for All Models
# ============================================================

# Prophet rent series: use raw rent values
df_rent_filtered["rent_for_prophet"] = df_rent_filtered["rent_value"]

# ARIMA rent series: use raw rent values (best and correct)
df_rent_filtered["rent_for_arima"] = df_rent_filtered["rent_value"]

# LSTM rent series: light smoothing (optional)
df_rent_filtered["rent_for_lstm"] = (
    df_rent_filtered["rent_value"]
    .rolling(window=2, center=True)
    .mean()
    .bfill()
    .ffill()
)

print(
    df_rent_filtered[
        ["date", "rent_value", "rent_for_prophet", "rent_for_arima", "rent_for_lstm"]
    ].head(12)
)


Interpolation Validation

We plot the raw annual rent values alongside the linear interpolation to confirm that the smoothed series is realistic and appropriate for ARIMA, which requires continuous monthly data.

In [1]:
df_rent_filtered["rent_diff"] = df_rent_filtered["rent_value"].diff()
plt.plot(df_rent_filtered["date"], df_rent_filtered["rent_diff"])
plt.title("First Difference of Rent Series")
plt.grid(True)
plt.show()


NameError: name 'df_rent_filtered' is not defined

In [ ]:
from statsmodels.tsa.stattools import adfuller

def adf_test(series, name="Series"):
    print(f"ADF Test for {name}")
    result = adfuller(series.dropna())

    labels = ["ADF Statistic", "p-value", "# Lags Used", "# Observations"]
    out = pd.Series(result[0:4], index=labels)

    for key, value in result[4].items():
        out[f"Critical Value ({key})"] = value

    print(out)
    
    if result[1] <= 0.05:
        print("=> Stationary (reject H0)\n")
    else:
        print("=> Non-stationary (fail to reject H0)\n")


In [ ]:
# ===========================
# ARIMA Feature Preparation
# ===========================

def create_arima_features(df, value_column):
    """
    Prepare a univariate, monthly time series for ARIMA/SARIMA modeling.

    Parameters
    ----------
    df : pandas.DataFrame
        Data filtered to a single city (and property type for HPI).
        Must contain columns ['date', value_column'].
    value_column : str
        The target column to model.

    Returns
    -------
    pandas.Series
        Clean monthly time series indexed by date and named 'value'.
    """
    ts = (
        df[["date", value_column]]
        .rename(columns={value_column: "value"})
        .set_index("date")
        .asfreq("MS")     # Monthly Start, required for SARIMA
        .sort_index()
    )

    return ts

# ===========================
# Calgary HPI (ARIMA-ready)
# ===========================

df_hpi_arima_filtered = df_hpi[
    (df_hpi["city"] == "Calgary") &
    (df_hpi["property_type"] == "All")
].copy()

hpi_arima_features = create_arima_features(
    df_hpi_arima_filtered,
    value_column="benchmark_price"
)

# ===========================
# Calgary Rent (ARIMA-ready)
# ===========================

df_rent_arima_filtered = df_rent_filtered[
    (df_rent_filtered["city"] == "Calgary")
].copy()

rent_arima_features = create_arima_features(
    df_rent_arima_filtered,
    value_column="rent_for_arima"   # raw rent series
)

hpi_arima_features.head(), rent_arima_features.head()


In [ ]:
from statsmodels.tsa.seasonal import STL
import matplotlib.pyplot as plt

def run_stl(series, title):
    """
    Runs STL decomposition on a univariate time series.
    
    Parameters
    ----------
    series : pandas.Series
        Time series indexed by date with monthly frequency.
    title : str
        Title to display on the STL plot.
    """
    stl = STL(series, robust=True)
    res = stl.fit()

    fig = res.plot()
    fig.set_size_inches(12, 8)
    fig.suptitle(title, fontsize=16)
    plt.show()


In [ ]:
run_stl(
    hpi_arima_features["value"],
    "STL Decomposition – Calgary HPI (benchmark_price)"
)


In [ ]:
run_stl(
    rent_arima_features["value"],
    "STL Decomposition – Calgary Rent (rent_value)"
)


# 📈 ARIMA Model Training & Evaluation

In this section we build, train, and evaluate ARIMA/SARIMA models for:

- Calgary House Price Index (HPI)
- Calgary Rent Index

We follow a structured forecasting workflow:

1. Prepare ARIMA-ready features  
2. Confirm stationarity using ADF  
3. Analyze trend and seasonality using STL  
4. Select model order using Auto-ARIMA (AIC/BIC optimization)  
5. Fit the final model  
6. Evaluate residuals  
7. Forecast future values  
8. Plot and interpret results  

HPI uses strong seasonal SARIMA.  
Rent uses non-seasonal ARIMA or very light SARIMA.


In [ ]:
# STEP 1 — Prepare ARIMA Features & Train/Test Split
def train_test_split(ts, split_date="2021-01-01"):
    train = ts[:split_date]
    test   = ts[split_date:]
    return train, test

# HPI Series
hpi_ts = hpi_arima_features["value"]
hpi_train, hpi_test = train_test_split(hpi_ts)

# Rent Series
rent_ts = rent_arima_features["value"]
rent_train, rent_test = train_test_split(rent_ts)

hpi_train.tail(), hpi_test.head()

In [ ]:
#STEP 2 — ADF (Stationarity Test)
plt.figure(figsize=(14,4))
from statsmodels.tsa.stattools import adfuller

def adf_report(series, name="Series"):
    result = adfuller(series.dropna())
    print(f"\nADF Test for {name}")
    print(f"ADF Statistic: {result[0]:.4f}")
    print(f"p-value: {result[1]:.6f}")
    for key, val in result[4].items():
        print(f"Critical Value ({key}): {val:.4f}")
    print("=> Stationary" if result[1] < 0.05 else "=> Non-stationary")

adf_report(hpi_train, "HPI Train")
adf_report(rent_train, "Rent Train")



In [ ]:
# STEP 3 — STL Seasonality Decomposition (Correct Visual)
from statsmodels.tsa.seasonal import STL
import matplotlib.pyplot as plt

def stl_plot(ts, title):
    stl = STL(ts, robust=True)
    res = stl.fit()
    fig = res.plot()
    fig.set_size_inches(12, 8)
    fig.suptitle(title, fontsize=16)
    plt.show()

stl_plot(hpi_train, "STL Decomposition — Calgary HPI")
stl_plot(rent_train, "STL Decomposition — Calgary Rent")



In [ ]:
# A) Auto-ARIMA for HPI — Seasonal SARIMA
from pmdarima import auto_arima

hpi_model = auto_arima(
    hpi_train,
    seasonal=True,
    m=12,                  # monthly seasonality
    d=1,                  # from ADF & differencing
    D=1,                  # seasonal differencing
    trace=True,
    error_action='ignore',
    suppress_warnings=True,
    stepwise=True
)

print(hpi_model.summary())


In [ ]:
# B) Auto-ARIMA for Rent — Weak/Minimal Seasonality
rent_model = auto_arima(
    rent_train,
    seasonal=True,
    m=12,
    d=1,
    D=0,              # based on STL seasonality (weak)
    trace=True,
    error_action='ignore',
    suppress_warnings=True,
    stepwise=True
)

print(rent_model.summary())


In [ ]:
# STEP 4 — Fit Final Models on Full Training Set
hpi_model.fit(hpi_train)
rent_model.fit(rent_train)


In [ ]:
# STEP 5 — Forecast on Test Set
hpi_pred = hpi_model.predict(n_periods=len(hpi_test))
rent_pred = rent_model.predict(n_periods=len(rent_test))


In [ ]:
# STEP 6 — Plot Actual vs Forecast (Test Set)
# HPI Plot
plt.figure(figsize=(14,6))
plt.plot(hpi_train.index, hpi_train, label="Training")
plt.plot(hpi_test.index, hpi_test, label="Actual")
plt.plot(hpi_test.index, hpi_pred, label="Forecast")
plt.title("HPI – ARIMA Forecast vs Actual")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Rent Plot
plt.figure(figsize=(14,6))
plt.plot(rent_train.index, rent_train, label="Training")
plt.plot(rent_test.index, rent_test, label="Actual")
plt.plot(rent_test.index, rent_pred, label="Forecast")
plt.title("Rent – ARIMA Forecast vs Actual")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# STEP 7 — Residual Diagnostics
hpi_model.plot_diagnostics(figsize=(12,8))
plt.show()

rent_model.plot_diagnostics(figsize=(12,8))
plt.show()


In [ ]:
# STEP 8 — Forecast Into the Future (12–36 months)
future_hpi = hpi_model.predict(n_periods=36)
future_rent = rent_model.predict(n_periods=36)


In [ ]:
# STEP 9 — Plot Future Forecasts
future_index_hpi = pd.date_range(start=hpi_arima_features.index[-1], periods=36+1, freq="MS")[1:]

plt.figure(figsize=(14,6))
plt.plot(hpi_arima_features.index, hpi_arima_features["value"], label="Historical")
plt.plot(future_index_hpi, future_hpi, label="Forecast", linestyle="--")
plt.title("HPI Forecast (36 Months Ahead)")
plt.legend()
plt.grid(True)
plt.show()


## Interpretation

### HPI Model (Seasonal SARIMA)
- The model captures strong yearly seasonality and long-term trend.
- Residuals behave as white noise.
- Forecast follows market dynamics and recent acceleration.

### Rent Model (ARIMA)
- Rent shows weak seasonality, so non-seasonal ARIMA is sufficient.
- Jumps correspond to CREA benchmark updates, not monthly volatility.
- Residuals are small and stable, indicating a good fit.

Overall, the ARIMA models perform well and provide credible, stable forecasting behavior.


In [ ]:
# ===========================
# Prophet Feature Preparation
# ===========================

def create_prophet_features(df, target_column, regressors=None):
    """
    Prepare a dataframe for Prophet forecasting with optional regressors.
    
    Parameters
    ----------
    df : pandas.DataFrame
        Filtered dataframe containing at least ['date', target_column].
    target_column : str
        The name of the column used as Prophet's target ('y').
    regressors : dict[str, pandas.DataFrame], optional
        A dictionary of regressor dataframes keyed by regressor name.
        Each must contain ['date', value].

    Returns
    -------
    pandas.DataFrame
        Prophet-ready dataframe with columns:
        ['ds', 'y', <regressors>]
    """
    # Base structure: ds + y
    feat = (
        df[["date", target_column]]
        .rename(columns={"date": "ds", target_column: "y"})
        .sort_values("ds")
        .reset_index(drop=True)
    )

    # Merge regressors if provided
    if regressors is not None:
        for reg_name, reg_df in regressors.items():
            reg_temp = reg_df.rename(columns={"date": "ds", reg_name: reg_name})
            feat = feat.merge(reg_temp[["ds", reg_name]], on="ds", how="left")

    return feat

# Example usage:
# prophet_features = create_prophet_features(df_hpi_filtered, "benchmark_price", regressors)



In [ ]:
# ===========================
# LSTM Feature Preparation
# ===========================

def create_lstm_features(df_price, df_rent, df_macro, df_demo):
    """
    Create a multivariate feature table for LSTM forecasting.
    Combines price, rent, macro, and demographic datasets,
    and adds engineered lag features.
    
    Parameters
    ----------
    df_price : pandas.DataFrame
        Price dataset with ['date', price_value].
    df_rent : pandas.DataFrame
        Rent dataset with ['date', rent_value].
    df_macro : pandas.DataFrame
        Macro dataset with columns like ['date', 'gdp_growth', 'cpi_yoy'].
    df_demo : pandas.DataFrame
        Demographic dataset with ['date', 'population', 'migration_rate'].

    Returns
    -------
    pandas.DataFrame
        A multivariate, monthly-frequency table with lag features.
    """

    # Merge everything on date
    df = (
        df_price
        .merge(df_rent, on="date", how="left", suffixes=("_price", "_rent"))
        .merge(df_macro, on="date", how="left")
        .merge(df_demo, on="date", how="left")
        .set_index("date")
        .asfreq("MS")
    )

    # Create standard lag features
    for lag in [1, 3, 6, 12]:
        df[f"lag_price_{lag}"] = df["benchmark_price"].shift(lag)

    # Example rolling mean features
    df["roll_mean_3"] = df["benchmark_price"].rolling(3).mean()
    df["roll_mean_6"] = df["benchmark_price"].rolling(6).mean()
    df["roll_mean_12"] = df["benchmark_price"].rolling(12).mean()

    # YoY change
    df["price_yoy"] = df["benchmark_price"].pct_change(12)

    return df

# Example usage:
# lstm_features = create_lstm_features(df_hpi_filtered, df_rent_filtered, df_macro, df_demo)



# Summary of EDA Findings

- HPI and Rent series are non-stationary (ADF)
- Strong seasonal structure detected (STL)
- Macro variables like interest rates, inflation, unemployment show meaningful correlation
- ARIMA requires differencing and univariate structure
- Prophet benefits from a few macro regressors
- LSTM requires a rich multivariate feature matrix
- Feature tables must be created separately for each model
